In [44]:
#import packages
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import re

In [41]:
#read in the csv file of elevated blood lead levels by census tract in CA
EBLL_df = pd.read_csv(rf'C:\Users\amarq\Python_Projects\raw_data\Lead Testing by CA Census Tract 2013 to 2018.csv')
EBLL_df.shape

(7632, 4)

In [43]:
#rename columns
EBLL_renaming = {'County - Census Tract' : 'Tract',
       'Number of Children Under Age 6 with Elevated Lead Levels':'Number_EBLL',
       'Number of Tests that Children in Medi-Cal Ages 1 and 2 Should have Received':'Number_MediCal',
       'Number of Tests that Children in Medi-Cal Ages 1 and 2 Missed':'Number_Not_Tested'}

EBLL_df.rename(columns=EBLL_renaming, inplace = True)


'Yolo County - Census Tract 106.07'

pandas.core.frame.DataFrame

In [52]:
#remove the words before the census tract designation

#use python .extract() method to only keep the  the tract number
tract_re = re.compile(r'\d*[.]?\d*?')

EBLL_df['Tract_Number'] = EBLL_df['Tract'].str.extract(r'\d*')


ValueError: pattern contains no capture groups

In [14]:
#bring in ACS Data
Age_of_Strucutre_df = pd.read_csv(rf'C:\Users\amarq\Python_Projects\raw_data\ACS_5Y_2018.B25126_AgeofStructureBuilt_Alameda County.csv')

In [15]:
EBLL_df.describe()

,Number of Children Under Age 6 with Elevated Lead Levels
count,7632.000000
mean,5.459251
std,7.717928
min,0.000000
25%,1.000000
50%,3.000000
75%,7.000000
max,153.000000


In [17]:
EBLL_df.columns

Index(['County - Census Tract',
       'Number of Children Under Age 6 with Elevated Lead Levels',
       'Number of Tests that Children in Medi-Cal Ages 1 and 2 Should have Received',
       'Number of Tests that Children in Medi-Cal Ages 1 and 2 Missed'],
      dtype='object')

In [ ]:
EBLL_df